## Read the dataset

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

df = pd.read_csv('hate_crime.csv')
df.head()


# cleaning the datset
## Replace Blank values with DataFrame.replace() methods.

In [ ]:
df2 = df.replace(r'^\s*$', np.NaN, regex=True)
df2.head()

#df['Tenant'].replace('', np.nan, inplace=True)
#print df

# Using DataFrame.mask() method to remove all blank values

In [ ]:
df22=df2.mask(df == '')
df22.head()

In [ ]:
#Dropping all na values
df_na = df22.dropna()
df_na.head()

In [ ]:
#Export the consolidated data to a new Excel workbook
df_na.to_excel("newexcel.xlsx")

## Read the newexcel data

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

df_newexcel = pd.read_excel('newexcel - Copy.xlsx')
df_newexcel.head()

## Aggregation and creating graphs

In [ ]:
import pandas as pd
import plotly.express as px


df_agg_1 = df_newexcel.groupby(['STATE_NAME','OFFENDER_RACE','MULTIPLE_OFFENSE'], 
                               as_index=False).agg({'TOTAL_INDIVIDUAL_VICTIMS':'sum'})

fig = px.bar(df_agg_1, x = 'OFFENDER_RACE', y = 'TOTAL_INDIVIDUAL_VICTIMS', color="OFFENDER_RACE")
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

df_agg_2 = df_newexcel.groupby(['OFFENDER_RACE', 'DATA_YEAR','MULTIPLE_OFFENSE'], as_index=False).agg({'TOTAL_OFFENDER_COUNT':'sum'})
fig = px.line(df_agg_2, x = 'DATA_YEAR', y = 'TOTAL_OFFENDER_COUNT', color="OFFENDER_RACE")
fig.show()

## Creating bar graph using crosstab

In [ ]:
# importing the pandas library
import pandas as pd
 
# Reading the csv file and storing it in a variable
import pandas as pd
import numpy as np
import plotly.express as px

df_newexcel = pd.read_excel('newexcel - Copy.xlsx')
# Creating crosstab
crosstb = pd.crosstab(df_newexcel.REGION_NAME, df_newexcel.MULTIPLE_OFFENSE)
 
# Creating barplot
barplot = crosstb.plot.bar(rot=0)

## creating sunburst graph 

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px


df_newexcel = pd.read_excel('newexcel - Copy.xlsx')

fig = px.sunburst(df_newexcel,
    path=['MULTIPLE_OFFENSE','REGION_NAME'],
    values='ADULT_VICTIM_COUNT',
    color = 'ADULT_OFFENDER_COUNT')

# S ----- Single offense type
# M-------Multiple offense type
    
fig.show()

# Crosstab with three variables

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px


df_newexcel = pd.read_excel('newexcel - Copy.xlsx')
crosstb = pd.crosstab(df_newexcel.MULTIPLE_OFFENSE, [df_newexcel.DATA_YEAR,
                               df_newexcel.OFFENDER_RACE])
 
# Bar plotting
a = crosstb.plot(kind='bar', rot=0)
a.legend(title='offense type in each year', bbox_to_anchor=(1, 1.02),
         loc='upper left')

In [ ]:
import pandas as pd
import plotly.express as px

my_color_scale = [(1, '#629B02'), (0.4, 'rgb(255,165,0)'), (1, '#7a214c')]

fig = px.density_heatmap(df_newexcel, x = 'OFFENDER_ETHNICITY', y = 'DATA_YEAR',
                         color_continuous_scale = my_color_scale)
fig.show()


In [ ]:
# create the scatterplot 
import pandas as pd
import plotly.express as px

fig = px.scatter(df_newexcel, x = "DATA_YEAR", y ="VICTIM_COUNT", size = "TOTAL_OFFENDER_COUNT", color = "OFFENDER_RACE",
                 #hover_name = "DIVISION_NAME",
                 size_max = 60)
fig.show()

In [ ]:
## Draw an area graph
fig = px.area(df_newexcel, x="DATA_YEAR", y="ADULT_OFFENDER_COUNT",color = "OFFENDER_RACE",line_group = "OFFENDER_ETHNICITY",
              hover_name = "OFFENDER_ETHNICITY",
              title='Juvenile victim per year')

fig.show()

In [ ]:
# create the pie chart
fig = px.pie(df_newexcel, values='JUVENILE_OFFENDER_COUNT', names='OFFENDER_RACE', title='Juvenile offender per race')
fig.show()

## Two input one output inline dashboard

In [ ]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input ,Output

#----------------------------------------------
df_newexcel = pd.read_excel('newexcel - Copy.xlsx')
df_agg_1 = df_newexcel.groupby(['STATE_NAME','OFFENDER_RACE','MULTIPLE_OFFENSE'], 
                               as_index=False).agg({'TOTAL_INDIVIDUAL_VICTIMS':'sum'})

# creating as jupyter dash object
app = JupyterDash(__name__, external_stylesheets= [dbc.themes.BOOTSTRAP])
# define rows and cols layout
row = html.Div([
    #label
   html.H3(children = 'States'),
            dbc.Row([
                dbc.Col(html.Div(dcc.Dropdown(id = 'dropdown_state',
                                              options = [{'label':i, 'value': i} for i in df_newexcel ['STATE_NAME'].unique()],
                                              value=df_newexcel['STATE_NAME'].iat[0],
                                             )#end of dropdown1
                                
                                ), width = 6),# end of col1
                #label
    html.H3(children = 'TOTAL INDIVIDUAL VICTIMS'),
            dbc.Row(dbc.Col(html.Div(dcc.Slider(
                                        id = 'Victims_slider',
                                        min = df_agg_1['TOTAL_INDIVIDUAL_VICTIMS'].min(),
                                        max = df_agg_1['TOTAL_INDIVIDUAL_VICTIMS'].max(),
                                        value = 100,
                                        marks = {str(TOTAL_INDIVIDUAL_VICTIMS): str(TOTAL_INDIVIDUAL_VICTIMS) for TOTAL_INDIVIDUAL_VICTIMS in df_agg_1['STATE_NAME'].unique()},
                                        step=50 
                                    ),
        
        ))),
               
                
                
                
            ]),# end of the row1
            dbc.Row([
                #dbc.Col(html.Div(dcc.Graph(id='line_chart_sub_category')), width = 6),
                dbc.Col(html.Div(dcc.Graph(id='bar_chart_state')), width = 14),
                
                
            ])# end of the row2
])# end of the html for the row


# use the layout in the application
app.layout = row

#----------------------------------------
#callback function for dropdown
@app.callback(Output('bar_chart_state', 'figure'),
             [Input('dropdown_state', 'value'), Input('Victims_slider', 'value')])
def update_bar_chart(victim_state, race_victim):

    df_filtered = df_agg_1.loc[(df_agg_1['STATE_NAME'] ==victim_state)& (df_agg_1['TOTAL_INDIVIDUAL_VICTIMS']>=(race_victim))]
    fig2_bar = px.bar(df_filtered, x = 'OFFENDER_RACE', y='TOTAL_INDIVIDUAL_VICTIMS', color="OFFENDER_RACE")
    return fig2_bar
#----------------------------------------


app.run_server(mode = 'inline')

## Two Input, Two Outputs

In [ ]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input ,Output

#----------------------------------------------
df_newexcel = pd.read_excel('newexcel - Copy.xlsx')
df_newexcel.head()

#-----------------------------------------------

# create a aggregated values
df_agg_1 = df_newexcel.groupby(['STATE_NAME', 'DATA_YEAR'], as_index=False).agg({'TOTAL_INDIVIDUAL_VICTIMS':'sum'})
df_agg_2 = df_newexcel.groupby(['OFFENDER_RACE', 'STATE_NAME'], as_index=False).agg({'TOTAL_OFFENDER_COUNT':'sum'})


#-----------------------------------------------

# creating as jupyter dash object
app = JupyterDash(__name__, external_stylesheets= [dbc.themes.BOOTSTRAP])
# define rows and cols layout
row = html.Div([
            dbc.Row([
                dbc.Col(html.H3("Total Individual Victims per year"), width = 6),
                dbc.Col(html.H3("Total Offender Count per Race"), width = 6),
                
                
            ]),
            dbc.Row([
                dbc.Col(html.Div(dcc.Dropdown(id = 'dropdown_state',
                                              options = [{'label':i, 'value': i} for i in df_newexcel['STATE_NAME'].unique()],
                                              value=df_newexcel['STATE_NAME'].iat[0],
                                             )#end of dropdown1
                                
                                ), width = 6),# end of col1
                dbc.Col(html.Div(dcc.Dropdown(id = 'dropdown_race',
                                              options = [{'label':i, 'value': i} for i in df_agg_2['OFFENDER_RACE'].unique()],
                                              value=df_agg_2['OFFENDER_RACE'].iat[0],
                                             )#end of dropdown2
                
                                ), width = 6),# end of col2
                
            ]),# end of the row
            dbc.Row([
                dbc.Col(html.Div(dcc.Graph(id='line_chart_state')), width = 6),
                dbc.Col(html.Div(dcc.Graph(id='bar_chart_race')), width = 6),
                
                
            ])# end of the row2
])# end of the html for the row

# end of the html for the row


# use the layout in the application
app.layout = row


#----------------------------------------
#callback function
@app.callback(Output('line_chart_state', 'figure'),
             Input('dropdown_state', 'value'))
def update_pie_chart(input_year):
    df_filtered_1 = df_agg_1[df_agg_1['STATE_NAME']==input_year]
    fig_line = px.line(df_filtered_1, x = 'DATA_YEAR', y='TOTAL_INDIVIDUAL_VICTIMS')
    fig_line.update_traces(line_color = 'green')
    return fig_line
#------------------------------------------------


#------------------------------------------------
#callback function for Bar 
@app.callback(Output('bar_chart_race', 'figure'),
             Input('dropdown_race', 'value'))
def update_bar_chart(input_race):
    df_filtered_2 = df_agg_2[(df_agg_2['OFFENDER_RACE'] ==input_race)]
    fig2_bar = px.bar(df_filtered_2, x = 'STATE_NAME', y='TOTAL_OFFENDER_COUNT',
                     color_discrete_sequence=px.colors.qualitative.Antique)
    return fig2_bar

#------------------------------------------------
app.run_server(mode = 'inline')

## creating dashboard 

In [ ]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input ,Output
import dash
import base64
import dash_html_components as html


#----------------------------------------------
df_newexcel = pd.read_excel('newexcel - Copy.xlsx')
# create a list of unique values
np_race = df_newexcel['OFFENDER_RACE'].unique()
np_race.view()

#-----------------------------------------------

# create a aggregated values
df_agg_1 = df_newexcel.groupby(['STATE_NAME', 'DATA_YEAR','MULTIPLE_OFFENSE'], as_index=False).agg({'TOTAL_INDIVIDUAL_VICTIMS':'sum'})
df_agg_2 = df_newexcel.groupby(['OFFENDER_RACE', 'STATE_NAME','MULTIPLE_OFFENSE','JUVENILE_OFFENDER_COUNT'], as_index=False).agg({'TOTAL_OFFENDER_COUNT':'sum'})


#-----------------------------------------------

# creating as jupyter dash object
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
# define rows and cols layout
row = html.Div([
            dbc.Row([
                dbc.Col(html.H3("Total Individual Victims per year"), width = 6),
                dbc.Col(html.H3("Total Offender Count per Race"), width = 6),
                
                
            ]),
            dbc.Row([
                dbc.Col(html.Div(dcc.Dropdown(id = 'dropdown_state',
                                              options = [{'label':i, 'value': i} for i in df_newexcel['STATE_NAME'].unique()],
                                              value=df_newexcel['STATE_NAME'].iat[0],
                                             )#end of dropdown1
                                
                                ), width = 6),# end of col1
                dbc.Col(html.Div(dcc.Dropdown(id = 'dropdown_race',
                                              options = [{'label':i, 'value': i} for i in df_agg_2['OFFENDER_RACE'].unique()],
                                              value=df_agg_2['OFFENDER_RACE'].iat[0],
                                             )#end of dropdown2
                
                                ), width = 6),# end of col2
                
            ]),# end of the row1
            dbc.Row([
                dbc.Col(html.Div(dcc.Graph(id='line_chart_state')), width = 6),
                dbc.Col(html.Div(dcc.Graph(id='bar_chart_race')), width = 6),
                
                
            ]),# end of the row2
            #label
           html.H3(children = 'States'),
            dbc.Row([
                dbc.Col(html.Div(dcc.Dropdown(id = 'dropdown_state1',
                                              options = [{'label':i, 'value': i} for i in df_newexcel ['STATE_NAME'].unique()],
                                              value=df_newexcel['STATE_NAME'].iat[0],
                                             )#end of dropdown1
                                
                                 ), width = 6),# end of col1
                
            ]),
            html.H3(children = 'TOTAL INDIVIDUAL VICTIMS'),
            dbc.Row([
                dbc.Col(html.Div(dcc.Slider(
                                        id = 'Victims_slider',
                                        min = df_agg_1['TOTAL_INDIVIDUAL_VICTIMS'].min(),
                                        max = df_agg_1['TOTAL_INDIVIDUAL_VICTIMS'].max(),
                                        value = 100,
                                        marks = {str(TOTAL_INDIVIDUAL_VICTIMS): str(TOTAL_INDIVIDUAL_VICTIMS) for TOTAL_INDIVIDUAL_VICTIMS in df_agg_1['STATE_NAME'].unique()},
                                        step=50 
                                    ),
                                )),
]),# end of the row2
            dbc.Row([
                dbc.Col(html.Div(dcc.Graph(id='bar_chart_state')), width = 14),
                
                
            ]),
            dbc.Row([
                dbc.Col(html.Div([html.H5("Offense in states"),
                    dcc.Checklist(options = {'S':  '   Single',
                                             'M':  '   Multiple'},
                                  value = ['S', 'M'], # setting what boxes should be checked
                                            labelStyle = dict(display = 'block'), # not in a row
                                            id = 'chk_SN')
                
                            ]), width = 3),# end of the column 1-1
            dbc.Col(html.Div(dcc.Graph(id = 'heatmap_count')), width = 11),# end of the column 1-2
            ]),# end of row1
            dbc.Row([
                dbc.Col(html.Div(), width = 3),# end of the column 2-1
                dbc.Col(html.Div(dcc.Graph(id = 'hist_offender')), width = 13),# end of the column 2-2
            ]),# end of row3
    
#create the Radiobuttons
    
            dbc.Row([
            dcc.RadioItems(id='race_rdb',
                  options = [{'value':i, 'label':i} for i in np_race],
                  value = np_race[0],
                  labelStyle= {'display': 'inline-block'}
                  ),
            dcc.Dropdown(id = 'state_dpdn', options = [0], multi = False),
            dcc.Graph(id = 'pie-chart')
    
])
    ])
# end of the html for the row

#----------------------------------------
#callback function
@app.callback(Output('line_chart_state', 'figure'),
             Input('dropdown_state', 'value'))
def update_pie_chart(input_year):
    df_filtered_1 = df_agg_1[df_agg_1['STATE_NAME']==input_year]
    fig_line = px.line(df_filtered_1, x = 'DATA_YEAR', y='TOTAL_INDIVIDUAL_VICTIMS')
    fig_line.update_traces(line_color = 'green')
    return fig_line
#------------------------------------------------


#------------------------------------------------
#callback function for Bar 
@app.callback(Output('bar_chart_race', 'figure'),
             Input('dropdown_race', 'value'))
def update_bar_chart(input_race):
    df_filtered_2 = df_agg_2[(df_agg_2['OFFENDER_RACE'] ==input_race)]
    fig2_bar = px.bar(df_filtered_2, x = 'STATE_NAME', y='TOTAL_OFFENDER_COUNT',
                     color_discrete_sequence=px.colors.qualitative.Antique)
    return fig2_bar
#------------------------------------------------

# two input one output
#----------------------------------------
#callback function for two input one output
@app.callback(Output('bar_chart_state', 'figure'),
             [Input('dropdown_state1', 'value'), Input('Victims_slider', 'value')])
def update_bar_chart(victim_state, race_victim):

    df_filtered = df_agg_2.loc[(df_agg_2['STATE_NAME'] ==victim_state)& (df_agg_2['TOTAL_OFFENDER_COUNT']>=(race_victim))]
    fig_bar = px.bar(df_filtered, x ='OFFENDER_RACE', y ='TOTAL_OFFENDER_COUNT', color="OFFENDER_RACE")
    return fig_bar
#----------------------------------------
# callback for one input two output

@app.callback(Output('heatmap_count', 'figure'),
              Output('hist_offender', 'figure'),
              Input('chk_SN', 'value'))
def update_heatmap(options_SN):
    df_filtered3 = df_agg_1.loc[df_agg_1['MULTIPLE_OFFENSE'].isin(options_SN)]
    df_filtered4 = df_agg_2.loc[df_agg_2['MULTIPLE_OFFENSE'].isin(options_SN)]
    fig_bar_s = px.bar(df_filtered3, x = 'STATE_NAME', y = 'TOTAL_INDIVIDUAL_VICTIMS', color="DATA_YEAR")
    #fig_line_s = px.line(df_agg_2, x = 'OFFENDER_ETHNICITY', y = 'TOTAL_OFFENDER_COUNT', color="OFFENDER_RACE")
    
    my_color_scale = [(1, '#629B02'), (0.4, 'rgb(255,165,0)'), (1, '#7a214c')]
    fig_heat = px.density_heatmap(df_filtered4, x = 'OFFENDER_RACE', y = 'TOTAL_OFFENDER_COUNT',
                         color_continuous_scale = my_color_scale)

    return fig_bar_s, fig_heat

# callback for radio buttons
@app.callback(
    Output('state_dpdn', 'options'),
    Input('race_rdb', 'value')
)
def set_state_options(chosen_race_rdb):
    df_filtered5 = df_newexcel[df_newexcel['OFFENDER_RACE']==chosen_race_rdb] # filter on category chosen by radio buttons
    dpdn_state = sorted(df_filtered5['STATE_NAME'].unique()) # create a list of sub-categories corresponding to the
    
    return [{'value':i, 'label':i} for i in dpdn_state]
# populate on initial value for sub-category dropdown box
@app.callback(
    Output('state_dpdn', 'value'),
    Input('state_dpdn', 'options')
)
def set_state_initial_value(available_options):
    return available_options[0]['value']
    
#callback for pie charts

#______________________________________

@app.callback(
    Output('pie-chart', 'figure'),
    [Input('race_rdb', 'value'),
     Input('state_dpdn', 'value')])
def update_pie_chart(sel_race, sel_state):
    
# filter the dataset based on the values chosen by the user

    df_filtered5 = df_newexcel[(df_newexcel['OFFENDER_RACE']==sel_race)& (df_newexcel['STATE_NAME']==sel_state)]

# create the aggregated data
    df_aggregated = df_filtered5.groupby(['DATA_YEAR'], as_index=False).agg({'ADULT_OFFENDER_COUNT':'sum'})

#create the pie chart showing the different segment as slices
    fig_pie = px.pie(df_aggregated, values = 'ADULT_OFFENDER_COUNT', names = 'DATA_YEAR')
    #fig = px.pie(df_newexcel, values='JUVENILE_OFFENDER_COUNT', names='OFFENDER_RACE', title='Juvenile offender per race')
    return fig_pie



app.layout = html.Div([row])
if __name__ == "__main__":
    app.run_server(debug = False)

In [ ]:
# image_filename = 'istockphoto-922764822-612x612.jpg' 
# encoded_image = base64.b64encode(open(image_filename, 'rb').read())
# print(encoded_image)
# html.Img(src='...'.format(encoded_image))
# html.Img(src='...'.format(encoded_image))


## create the dashboard using tab

In [ ]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input ,Output
import dash
import base64
import dash_html_components as html


#----------------------------------------------
df_newexcel = pd.read_excel('newexcel - Copy.xlsx')
# create a list of unique values
np_race = df_newexcel['OFFENDER_RACE'].unique()
np_race.view()

#-----------------------------------------------

# create a aggregated values
df_agg_1 = df_newexcel.groupby(['STATE_NAME', 'DATA_YEAR','MULTIPLE_OFFENSE'], as_index=False).agg({'TOTAL_INDIVIDUAL_VICTIMS':'sum'})
df_agg_2 = df_newexcel.groupby(['OFFENDER_RACE', 'STATE_NAME','MULTIPLE_OFFENSE','JUVENILE_OFFENDER_COUNT'], as_index=False).agg({'TOTAL_OFFENDER_COUNT':'sum'})


#-----------------------------------------------

# creating a dashboard object
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
image_filename = 'istockphoto-922764822-612x612.jpg' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


# define rows and cols layout
row = html.Div([
            dbc.Row([
                dbc.Col(html.H3("Total Individual Victims per year in each state"), width = 5),
                dbc.Col(html.H3("Total Offender Count per Race in each state"), width = 5),
                
                
            ]),
            dbc.Row([
                dbc.Col(html.Div(dcc.Dropdown(id = 'dropdown_state',
                                              options = [{'label':i, 'value': i} for i in df_newexcel['STATE_NAME'].unique()],
                                              value=df_newexcel['STATE_NAME'].iat[0],
                                             )#end of dropdown1
                                
                                ), width = 6),# end of col1
                dbc.Col(html.Div(dcc.Dropdown(id = 'dropdown_race',
                                              options = [{'label':i, 'value': i} for i in df_agg_2['OFFENDER_RACE'].unique()],
                                              value=df_agg_2['OFFENDER_RACE'].iat[0],
                                             )#end of dropdown2
                
                                ), width = 6),# end of col2
                
            ]),# end of the row1
            dbc.Row([
                dbc.Col(html.Div(dcc.Graph(id='line_chart_state')), width = 5),
                dbc.Col(html.Div(dcc.Graph(id='bar_chart_race')), width = 5),
                
                
            ]),# end of the row2
            #label
           html.H3(children = 'Select the states'),
            dbc.Row([
                dbc.Col(html.Div(dcc.Dropdown(id = 'dropdown_state1',
                                              options = [{'label':i, 'value': i} for i in df_newexcel ['STATE_NAME'].unique()],
                                              value=df_newexcel['STATE_NAME'].iat[0],
                                             )#end of dropdown1
                                
                                 ), width = 4),# end of col1
                
            ]),
            html.H3(children = 'Select the individual victims numbers you want to see'),
            dbc.Row([
                dbc.Col(html.Div(dcc.Slider(
                                        id = 'Victims_slider',
                                        min = df_agg_1['TOTAL_INDIVIDUAL_VICTIMS'].min(),
                                        max = df_agg_1['TOTAL_INDIVIDUAL_VICTIMS'].max(),
                                        value = 100,
                                        marks = {str(TOTAL_INDIVIDUAL_VICTIMS): str(TOTAL_INDIVIDUAL_VICTIMS) for TOTAL_INDIVIDUAL_VICTIMS in df_agg_1['STATE_NAME'].unique()},
                                        step=20 
                                    ),
                                )),
]),# end of the row2
            dbc.Row([
                dbc.Col(html.Div(dcc.Graph(id='bar_chart_state')), width = 14),
                
                
            ]),
            dbc.Row([
                dbc.Col(html.Div([html.H5("Category of Offense in states"),
                    dcc.Checklist(options = {'S':  '   Single',
                                             'M':  '   Multiple'},
                                  value = ['S', 'M'], # setting what boxes should be checked
                                            labelStyle = dict(display = 'block'), # not in a row
                                            id = 'chk_SN')
                
                            ]), width = 3),# end of the column 1-1
            dbc.Col(html.Div(dcc.Graph(id = 'heatmap_count')), width = 11),# end of the column 1-2
            ]),# end of row1
            dbc.Row([
                dbc.Col(html.Div(), width = 3),# end of the column 2-1
                dbc.Col(html.Div(dcc.Graph(id = 'hist_offender')), width = 13),# end of the column 2-2
            ]),# end of row3
    
#create the Radiobuttons
    
            dbc.Row([
            html.H2(children = 'Total individual victims in each state according to Race and years'),
            dcc.RadioItems(id='race_rdb',
                  options = [{'value':i, 'label':i} for i in np_race],
                  value = np_race[0],
                  labelStyle= {'display': 'inline-block'}
                  ),
            dcc.Dropdown(id = 'state_dpdn', options = [0], multi = False),
            dcc.Graph(id = 'pie-chart')
    
]),
    # layout and templates
            
            dbc.Row([
                dbc.Col(html.H2("Select the offense category"), width = 6),
                dbc.Tabs([
                dbc.Tab([
                html.Ul([
               html.Br(),
               html.Li('Individual'),
               html.Li('Government'),
               html.Li('Society/Public'),
               html.Li('Business'),
               html.Li([
                   'Source: ',
                   html.A('https://https://www.rover.com/blog/cute-cat-names/',
                          href='https://https://www.rover.com/blog/cute-cat-names/')
               ])
           ])
            

       ], label='Single'),
            dbc.Tab([
            html.Ul([
                html.Br(),
                html.Li('Business;Government'),
                html.Li('Business;Society/Public'),
                html.Li('Government;Individual'),
                html.Li('Business;Government;Individual'),
                html.Li(['Religious Organization: ',
                         html.A('https://www.istockphoto.com/photo/young-woman-with-disgusted-expression-repulsing-something-isolated-on-the-pink-gm922764822-253308047',
                                href='https://www.istockphoto.com/photo/young-woman-with-disgusted-expression-repulsing-something-isolated-on-the-pink-gm922764822-253308047')
                         ]),
               #html.Img(src='data:image/jpg;base64,https://media.istockphoto.com/id/922764822/photo/young-woman-with-disgusted-expression-repulsing-something-isolated-on-the-pink.jpg?s=612x612&w=is&k=20&c=cEek3J4cImtv8VomXvwsMDp-0dXG6eEnycHNE82Zo6E='.format(encoded_image))
               html.Img(src='...'.format(encoded_image)) 
            ])
        ], label='Multiple')
    ]),
])
            
])   
    
# end of the html for the row

#----------------------------------------
#callback function
@app.callback(Output('line_chart_state', 'figure'),
             Input('dropdown_state', 'value'))
def update_pie_chart(input_year):
    df_filtered_1 = df_agg_1[df_agg_1['STATE_NAME']==input_year]
    fig_line = px.line(df_filtered_1, x = 'DATA_YEAR', y='TOTAL_INDIVIDUAL_VICTIMS')
    fig_line.update_traces(line_color = 'green')
    return fig_line
#------------------------------------------------


#------------------------------------------------
#callback function for Bar 
@app.callback(Output('bar_chart_race', 'figure'),
             Input('dropdown_race', 'value'))
def update_bar_chart(input_race):
    df_filtered_2 = df_agg_2[(df_agg_2['OFFENDER_RACE'] ==input_race)]
    fig2_bar = px.bar(df_filtered_2, x = 'STATE_NAME', y='TOTAL_OFFENDER_COUNT',
                     color_discrete_sequence=px.colors.qualitative.Antique)
    return fig2_bar
#------------------------------------------------

# two input one output
#----------------------------------------
#callback function for two input one output
@app.callback(Output('bar_chart_state', 'figure'),
             [Input('dropdown_state1', 'value'), Input('Victims_slider', 'value')])
def update_bar_chart(victim_state, race_victim):

    df_filtered = df_agg_2.loc[(df_agg_2['STATE_NAME'] ==victim_state)& (df_agg_2['TOTAL_OFFENDER_COUNT']>=(race_victim))]
    fig_bar = px.bar(df_filtered, x ='OFFENDER_RACE', y ='TOTAL_OFFENDER_COUNT', color="OFFENDER_RACE")
    return fig_bar
#----------------------------------------
# callback for one input two output

@app.callback(Output('heatmap_count', 'figure'),
              Output('hist_offender', 'figure'),
              Input('chk_SN', 'value'))
def update_heatmap(options_SN):
    df_filtered3 = df_agg_1.loc[df_agg_1['MULTIPLE_OFFENSE'].isin(options_SN)]
    df_filtered4 = df_agg_2.loc[df_agg_2['MULTIPLE_OFFENSE'].isin(options_SN)]
    fig_bar_s = px.bar(df_filtered3, x = 'STATE_NAME', y = 'TOTAL_INDIVIDUAL_VICTIMS', color="DATA_YEAR")
    #fig_line_s = px.line(df_agg_2, x = 'OFFENDER_ETHNICITY', y = 'TOTAL_OFFENDER_COUNT', color="OFFENDER_RACE")
    
    my_color_scale = [(1, '#629B02'), (0.4, 'rgb(255,165,0)'), (1, '#7a214c')]
    fig_heat = px.density_heatmap(df_filtered4, x = 'OFFENDER_RACE', y = 'TOTAL_OFFENDER_COUNT',
                         color_continuous_scale = my_color_scale)

    return fig_bar_s, fig_heat

# callback for radio buttons
@app.callback(
    Output('state_dpdn', 'options'),
    Input('race_rdb', 'value')
)
def set_state_options(chosen_race_rdb):
    df_filtered5 = df_newexcel[df_newexcel['OFFENDER_RACE']==chosen_race_rdb] # filter on category chosen by radio buttons
    dpdn_state = sorted(df_filtered5['STATE_NAME'].unique()) # create a list of sub-categories corresponding to the
    
    return [{'value':i, 'label':i} for i in dpdn_state]
# populate on initial value for sub-category dropdown box
@app.callback(
    Output('state_dpdn', 'value'),
    Input('state_dpdn', 'options')
)
def set_state_initial_value(available_options):
    return available_options[0]['value']
    
#callback for pie charts

#______________________________________

@app.callback(
    Output('pie-chart', 'figure'),
    [Input('race_rdb', 'value'),
     Input('state_dpdn', 'value')])
def update_pie_chart(sel_race, sel_state):
    
# filter the dataset based on the values chosen by the user

    df_filtered5 = df_newexcel[(df_newexcel['OFFENDER_RACE']==sel_race)& (df_newexcel['STATE_NAME']==sel_state)]

# create the aggregated data
    df_aggregated = df_filtered5.groupby(['DATA_YEAR'], as_index=False).agg({'ADULT_OFFENDER_COUNT':'sum'})

#create the pie chart showing the different segment as slices
    fig_pie = px.pie(df_aggregated, values = 'ADULT_OFFENDER_COUNT', names = 'DATA_YEAR')
    #fig = px.pie(df_newexcel, values='JUVENILE_OFFENDER_COUNT', names='OFFENDER_RACE', title='Juvenile offender per race')
    return fig_pie



app.layout = html.Div([row])
if __name__ == "__main__":
    app.run_server(debug = False)

## Inline dashboard using chained callback

In [ ]:
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc


df = pd.read_excel('newexcel.xlsx')
df = df.sort_values(by=['REGION_NAME'])

# create a list of unique values
np_state = df['REGION_NAME'].unique()
np_state.view()


app = JupyterDash(__name__)
app.layout = html.Div([
    dcc.RadioItems(id='region_rdb',
                  options = [{'value':i, 'label':i} for i in np_state],
                  value = np_state[0],
                  labelStyle= {'display': 'inline-block'}
                  ),
    
    dcc.Dropdown(id = 'state_dpdn', options = [0], multi = False),
    dcc.Graph(id = 'bar_chart'),
    dcc.Graph(id = 'scatter_offender')    
]) 
            
@app.callback(
    Output('state_dpdn', 'options'),
    Input('region_rdb', 'value')
)
def set_state_options(chosen_state_rdb):
    df_filtered = df[df['REGION_NAME']==chosen_state_rdb] 
    dpdn_region = sorted(df_filtered['STATE_NAME'].unique()) 
    
    return [{'value':i, 'label':i} for i in dpdn_region]

@app.callback(
    Output('state_dpdn', 'value'),
    Input('state_dpdn', 'options')
)
def set_city_initial_value(available_options):
    return available_options[0]['value']
    
#callback for graphs

@app.callback(Output('bar_chart', 'figure'),
              Output('scatter_offender', 'figure'),
              Input('region_rdb', 'value'))
def update_bar_chart(state_crime):

    df_filtered = df[(df['REGION_NAME']==state_crime)]
    
    #& (df['OFFENDER_RACE']==sel_city)
    
    df_agg1 = df_filtered.groupby(['STATE_NAME','OFFENDER_RACE','DATA_YEAR'], as_index=False).agg({'TOTAL_INDIVIDUAL_VICTIMS':'sum'})
    df_agg2 = df_filtered.groupby(['OFFENDER_RACE'], as_index=False).agg({'TOTAL_OFFENDER_COUNT':'sum'})

    fig_bar = px.bar(df_agg1, x = 'DATA_YEAR', y = 'TOTAL_INDIVIDUAL_VICTIMS', color="OFFENDER_RACE")
    

    fig_scatter = px.scatter(df_filtered, x="ADULT_VICTIM_COUNT", y="ADULT_OFFENDER_COUNT",
        size="VICTIM_COUNT", color="OFFENDER_RACE",
                 hover_name="STATE_NAME", size_max=60)

    return fig_bar,fig_scatter
    
if __name__ == '__main__':
    app.run_server(mode='inline')

## Single and multiple offense graph that is represented as S and M in dataset

## Inserting graphs in Tab single and multiple

In [ ]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input ,Output
import dash
import base64
import dash_html_components as html


#----------------------------------------------
df_newexcel = pd.read_excel('newexcel - Copy.xlsx')
# create a list of unique values
np_race = df_newexcel['OFFENDER_RACE'].unique()
np_race.view()

#-----------------------------------------------

# create a aggregated values
df_agg_1 = df_newexcel.groupby(['STATE_NAME', 'DATA_YEAR','MULTIPLE_OFFENSE'], as_index=False).agg({'TOTAL_INDIVIDUAL_VICTIMS':'sum'})
df_agg_2 = df_newexcel.groupby(['OFFENDER_RACE', 'STATE_NAME','MULTIPLE_OFFENSE','JUVENILE_OFFENDER_COUNT'], as_index=False).agg({'TOTAL_OFFENDER_COUNT':'sum'})


#-----------------------------------------------

# creating a dashboard object
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
image_filename = 'istockphoto-922764822-612x612.jpg' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
image_filename2 = 'Screenshot 2022-12-16 154916.png'
encoded_image2 = base64.b64encode(open(image_filename2, 'rb').read())


# define rows and cols layout
row = html.Div([
            dbc.Row([
                dbc.Col(html.H3("Total Individual Victims per year in each state"), width = 5),
                dbc.Col(html.H3("Total Offender Count per Race in each state"), width = 5),
                
                
            ]),
            dbc.Row([
                dbc.Col(html.Div(dcc.Dropdown(id = 'dropdown_state',
                                              options = [{'label':i, 'value': i} for i in df_newexcel['STATE_NAME'].unique()],
                                              value=df_newexcel['STATE_NAME'].iat[0],
                                             )#end of dropdown1
                                
                                ), width = 6),# end of col1
                dbc.Col(html.Div(dcc.Dropdown(id = 'dropdown_race',
                                              options = [{'label':i, 'value': i} for i in df_agg_2['OFFENDER_RACE'].unique()],
                                              value=df_agg_2['OFFENDER_RACE'].iat[0],
                                             )#end of dropdown2
                
                                ), width = 6),# end of col2
                
            ]),# end of the row1
            dbc.Row([
                dbc.Col(html.Div(dcc.Graph(id='line_chart_state')), width = 5),
                dbc.Col(html.Div(dcc.Graph(id='bar_chart_race')), width = 5),
                
                
            ]),# end of the row2
            #label
           html.H3(children = 'Select the states'),
            dbc.Row([
                dbc.Col(html.Div(dcc.Dropdown(id = 'dropdown_state1',
                                              options = [{'label':i, 'value': i} for i in df_newexcel ['STATE_NAME'].unique()],
                                              value=df_newexcel['STATE_NAME'].iat[0],
                                             )#end of dropdown1
                                
                                 ), width = 4),# end of col1
                
            ]),
            html.H3(children = 'Select the individual victims numbers you want to see'),
            dbc.Row([
                dbc.Col(html.Div(dcc.Slider(
                                        id = 'Victims_slider',
                                        min = df_agg_1['TOTAL_INDIVIDUAL_VICTIMS'].min(),
                                        max = df_agg_1['TOTAL_INDIVIDUAL_VICTIMS'].max(),
                                        value = 20,
                                        marks = {str(TOTAL_INDIVIDUAL_VICTIMS): str(TOTAL_INDIVIDUAL_VICTIMS) for TOTAL_INDIVIDUAL_VICTIMS in df_agg_1['STATE_NAME'].unique()},
                                        step=50 
                                    ),
                                )),
]),# end of the row2
            dbc.Row([
                dbc.Col(html.Div(dcc.Graph(id='bar_chart_state')), width = 14),
                
                
            ]),
            dbc.Row([
                dbc.Col(html.Div([html.H5("Category of Offense in states"),
                    dcc.Checklist(options = {'S':  '   Single',
                                             'M':  '   Multiple'},
                                  value = ['S', 'M'], # setting what boxes should be checked
                                            labelStyle = dict(display = 'block'), # not in a row
                                            id = 'chk_SN')
                
                            ]), width = 3),# end of the column 1-1
            dbc.Col(html.Div(dcc.Graph(id = 'heatmap_count')), width = 11),# end of the column 1-2
            ]),# end of row1
            dbc.Row([
                dbc.Col(html.Div(), width = 3),# end of the column 2-1
                dbc.Col(html.Div(dcc.Graph(id = 'hist_offender')), width = 13),# end of the column 2-2
            ]),# end of row3
    
#create the Radiobuttons
    
            dbc.Row([
            html.H2(children = 'Total individual victims in each state according to Race and years'),
            dcc.RadioItems(id='race_rdb',
                  options = [{'value':i, 'label':i} for i in np_race],
                  value = np_race[0],
                  labelStyle= {'display': 'inline-block'}
                  ),
            dcc.Dropdown(id = 'state_dpdn', options = [0], multi = False),
            dcc.Graph(id = 'pie-chart')
    
]),
    # layout and templates
            
            dbc.Row([
                dbc.Col(html.H2("Select the offense category"), width = 6),
                dbc.Tabs([
                dbc.Tab([
                html.Ul([
               html.Br(),
               html.Li('Individual'),
               html.Li('Government'),
               html.Li('Society/Public'),
               html.Li('Business'),
               html.Li(['Source: ',
                   html.A('http://localhost:8888/notebooks/Downloads/Untitled32.ipynb?kernel_name=python3#',
                          href='http://localhost:8888/notebooks/Downloads/Untitled32.ipynb?kernel_name=python3#')
               ]),
               html.Img(src='...'.format(encoded_image2)) 
            
            ])
            

       ], label='Single'),
            dbc.Tab([
            html.Ul([
                html.Br(),
                html.Li('Business;Government'),
                html.Li('Business;Society/Public'),
                html.Li('Government;Individual'),
                html.Li('Business;Government;Individual'),
                html.Li(['Religious Organization: ',
                         html.A('http://localhost:8888/notebooks/Downloads/Untitled32.ipynb?kernel_name=python3#',
                                href='http://localhost:8888/notebooks/Downloads/Untitled32.ipynb?kernel_name=python3#')
                         ]),
               #html.Img(src='data:image/jpg;base64,https://media.istockphoto.com/id/922764822/photo/young-woman-with-disgusted-expression-repulsing-something-isolated-on-the-pink.jpg?s=612x612&w=is&k=20&c=cEek3J4cImtv8VomXvwsMDp-0dXG6eEnycHNE82Zo6E='.format(encoded_image))
               html.Img(src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAXcAAAEHCAYAAABV4gY/AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjMuNCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8QVMy6AAAACXBIWXMAAAsTAAALEwEAmpwYAAAc0klEQVR4nO3de5hU1Z3u8e8LEiBpR+Wih4AjTIIJELWjHYlJ9LQ6I6gnmhiNGE10NJJJMF6IiTqahEnkOegZ43iOmojGwHhDkigh3glHEi94aZSgYBxQmdBK5KKJMCIG/M0fe3VT3VR3V3dXdTeb9/M89dSutdfetWp31VurV+1apYjAzMzypVd3N8DMzMrP4W5mlkMOdzOzHHK4m5nlkMPdzCyHdunuBgAMGjQohg8f3t3NMDPboSxatGhdRAwutq7NcJfUD/gd0DfV/0VEfF/SAOBOYDiwEvhiRLyZtrkEOAvYCpwbEQ+2dh/Dhw+nrq6u5AdkZmYg6T9bWlfKsMxm4IiIOACoBsZL+iRwMTA/IkYC89NtJI0GJgBjgPHA9ZJ6d+oRmJlZu7QZ7pHZmG72SZcAjgdmpvKZwOfS8vHArIjYHBGvACuAg8vZaDMza11JH6hK6i1pMbAGmBcRTwJ7RcRqgHS9Z6o+FFhVsHl9Kmu+z4mS6iTVrV27thMPwczMmivpA9WI2ApUS9oduFvSx1qprmK7KLLP6cB0gJqaGs+BYNaN/vrXv1JfX88777zT3U2xIvr168ewYcPo06dPydu062yZiPizpAVkY+mvSxoSEaslDSHr1UPWU9+7YLNhwGvtuR8z61r19fXsuuuuDB8+HKlY/8y6S0Swfv166uvrGTFiRMnbtTksI2lw6rEjqT/w98AfgLnA6ana6cCv0vJcYIKkvpJGACOBp0pukZl1uXfeeYeBAwc62HsgSQwcOLDd/1WV0nMfAsxMZ7z0AmZHxD2SFgKzJZ0F/BE4CSAilkqaDSwDtgCT0rCOmfVgDvaeqyN/mzbDPSKWAB8vUr4eOLKFbaYCU9vdGjMzKwtPP2BmlkM9YvqBHmPKbmXYx186vw+zHkASp512GrfccgsAW7ZsYciQIYwdO5Z77rmHKVOmUFVVxYUXXti4TcO3zQcNGkRVVRUbN2ZfkRk7diybN2/mjTfeYNOmTQwdmp0dPWfOHGpraxu36d27N/vttx9btmxh1KhRzJw5k/e///1N9tVgypQp3HjjjQwevO3b9wsWLGD33Xcv+ngeffRRJk+ezFtvvQXA5MmTmThxYtF9jR8/nmnTplFbW8vq1avp378/AJdddhknnnhiYzsbTJgwgYsvvpja2lo2btzY+I37uro6LrzwQhYsWMDbb7/N2WefzZIlS4gIdt99dx544AGqqqpa3F9nONzNrKgPfOADPP/882zatIn+/fszb968xlBuryeffBKAGTNmUFdXx7XXXlu0Xv/+/Vm8eDEAp556Kj/5yU+YPHlyi/u94IILmry5tORPf/oTX/rSl5gzZw4HHngg69atY9y4cQwdOpRjjz221X3ddttt1NTUtNjO5tasWcP999/P0Ucf3aT8mmuuYa+99uK5554D4MUXX2w8tbG1/XWUh2XMrEVHH3009957LwB33HEHp5xySpfd96GHHsqKFSvKsq/rrruOM844gwMPPBCAQYMGceWVVzJt2rSy7L/Qt7/9bS6//PLtylevXt3kzfEjH/kIffv2Lfv9N3C4m1mLJkyYwKxZs3jnnXdYsmQJY8eO7ZL73bJlC/fff3+ToYpirr76aqqrq6murubwww9vsd7SpUs56KCDmpTV1NSwdOnSovt68MFtcx2eeuqpjeXr168HYNOmTY1l1dXV3HnnnY31DznkEPr27cvDDz/c5P7OPPNMrrjiCg455BAuu+wyli9f3riutf11lIdlzKxF+++/PytXruSOO+7gmGOOabKupdPzOnNKZUPIQdZzP+uss1qtX+qwTEQUbVdhWbmGZSAbm7/88su54oorGsuqq6t5+eWXeeihh/jNb37DJz7xCRYuXMioUaMqMizjcDezVh133HGNHwo29FwBBg4cyOrVq5vU3bBhQ4sfaJaiEiEHMGbMGOrq6jjuuOMayxYtWsTo0aPLfl8ARxxxBN/97nd54oknmpRXVVVxwgkncMIJJ9CrVy/uu+8+Ro0aVZE2eFjGzFp15pln8r3vfW+7IZLDDjuMuXPnsmHDBgDuuusuDjjgAHr37nkzfE+aNIkZM2Y0vnGsX7+eiy66iO985zsVu89LL72UK6+8svH2Y489xptvvgnAu+++y7Jly9hnn30qdv/uuZtZq4YNG8Z55523Xfn+++/POeecw2c+8xkkseeee3LTTTc1rn/77bcZNmxY4+3JkyczYMCADrWh2L4gGye/9dZbG8vnzJlDsV91GzJkCLfeeitnn302GzZsICI4//zz+exnP9uh9hQOH8G2UycLHXPMMU1O03zppZf4+te/TkTw3nvvceyxx/KFL3yh5P21lyK6f0LGmpqa6BG/xOTz3G0n9cILL1RseMDKo9jfSNKiiKgpVt89d7O2dPZN32/41g0c7maWGw8++CAXXXRRk7IRI0Zw9913d1OLuo/D3cxyY9y4cYwbN667m9Ej+GwZM7MccribmeWQw93MLIc85m5mPcbwi+8t6/5WTju2zTptTW28o3LP3cx2aoVTGwOdmtq4J3G4m9lOrzunNq4Uh7uZ7fS6a2rjSnK4m9lOr7WpjXdU/kDVzIyWpzbeUTnczczIpjbebbfd2G+//ViwYEF3N6fTHO5m1mOUcupipbQ0tfGOyuFuZju1jRs3bldWW1tLbW1t1zemjPyBqplZDjnczcxyqM1wl7S3pIclvSBpqaTzUvkUSa9KWpwuxxRsc4mkFZJelOT5N83MulgpY+5bgG9FxDOSdgUWSZqX1l0dEf9aWFnSaGACMAb4IPAbSftGxNZyNtzMzFrWZs89IlZHxDNpeQPwAtDaxAvHA7MiYnNEvAKsAA4uR2PNzKw07RpzlzQc+DjwZCo6R9ISSTdL2iOVDQVWFWxWT5E3A0kTJdVJqlu7dm37W25mZi0q+VRISVXAL4HzI+ItST8GfghEur4KOBNQkc1ju4KI6cB0gJqamu3Wm9lOqLM/Rr7d/kr7cfKpU6dy++2307t3b3r16sUNN9yww88vU1K4S+pDFuy3RcRdABHxesH6G4GGiY/rgb0LNh8GvFaW1pqZldnChQu55557eOaZZ+jbty/r1q3j3Xff7e5mdVopZ8sI+CnwQkT8qKB8SEG1zwPPp+W5wARJfSWNAEYCT5WvyWZm5bN69WoGDRpE3759ARg0aBAf/OAHu7lVnVfKmPungS8DRzQ77fFKSc9JWgIcDlwAEBFLgdnAMuABYJLPlDGznuqoo45i1apV7LvvvnzjG9/gt7/9bXc3qSzaHJaJiEcpPo5+XyvbTAWmdqJdZmZdoqqqikWLFvHII4/w8MMPc/LJJzNt2jTOOOOM7m5ap3huGTPb6fXu3btxPpn99tuPmTNn7vDh7ukHzGyn9uKLL7J8+fLG24sXL2afffbpxhaVh3vuZtZzlHjqYjlt3LiRb37zm/z5z39ml1124cMf/jDTp0/v8naUm8PdzHZqBx10EI8//nh3N6PsPCxjZpZDDnczsxxyuJsZABGeBaSn6sjfxuFuZvTr14/169c74HugiGD9+vX069evXdv5A1UzY9iwYdTX1+MZWnumfv36MWzYsHZt43A3M/r06cOIESO6uxlWRh6WMTPLIYe7mVkOOdzNzHLI4W5mlkMOdzOzHHK4m5nlkMPdzCyHHO5mZjnkcDczyyGHu5lZDjnczcxyyOFuZpZDDnczsxxyuJuZ5ZDD3cwshxzuZmY51Ga4S9pb0sOSXpC0VNJ5qXyApHmSlqfrPQq2uUTSCkkvShpXyQdgZmbbK6XnvgX4VkSMAj4JTJI0GrgYmB8RI4H56TZp3QRgDDAeuF5S70o03szMimsz3CNidUQ8k5Y3AC8AQ4HjgZmp2kzgc2n5eGBWRGyOiFeAFcDBZW63mZm1ol1j7pKGAx8HngT2iojVkL0BAHumakOBVQWb1acyMzPrIiWHu6Qq4JfA+RHxVmtVi5RFkf1NlFQnqc6/uG5mVl4lhbukPmTBfltE3JWKX5c0JK0fAqxJ5fXA3gWbDwNea77PiJgeETURUTN48OCOtt/MzIoo5WwZAT8FXoiIHxWsmgucnpZPB35VUD5BUl9JI4CRwFPla7KZmbVllxLqfBr4MvCcpMWp7J+BacBsSWcBfwROAoiIpZJmA8vIzrSZFBFby91wMzNrWZvhHhGPUnwcHeDIFraZCkztRLvMzKwT/A1VM7MccribmeWQw93MLIcc7mZmOeRwNzPLIYe7mVkOOdzNzHLI4W5mlkMOdzOzHHK4m5nlkMPdzCyHHO5mZjnkcDczyyGHu5lZDjnczcxyyOFuZpZDDnczsxxyuJuZ5ZDD3cwshxzuZmY55HA3M8uhXbq7AWa2A5myWye3/0t52mFtcs/dzCyHHO5mZjnkcDczyyGHu5lZDjnczcxyqM1wl3SzpDWSni8omyLpVUmL0+WYgnWXSFoh6UVJ4yrVcDMza1kpPfcZwPgi5VdHRHW63AcgaTQwARiTtrleUu9yNdbMzErTZrhHxO+AN0rc3/HArIjYHBGvACuAgzvRPjMz64DOjLmfI2lJGrbZI5UNBVYV1KlPZduRNFFSnaS6tWvXdqIZZmbWXEfD/cfAh4BqYDVwVSpXkbpRbAcRMT0iaiKiZvDgwR1shpmZFdOhcI+I1yNia0S8B9zItqGXemDvgqrDgNc610QzM2uvDoW7pCEFNz8PNJxJMxeYIKmvpBHASOCpzjXRzMzaq82JwyTdAdQCgyTVA98HaiVVkw25rAS+BhARSyXNBpYBW4BJEbG1Ii03M7MWtRnuEXFKkeKftlJ/KjC1M40yM7PO8TdUzcxyyOFuZpZDDnczsxxyuJuZ5ZDD3cwshxzuZmY55HA3M8shh7uZWQ453M3McsjhbmaWQw53M7MccribmeWQw93MLIcc7mZmOeRwNzPLIYe7mVkOOdzNzHLI4W5mlkMOdzOzHHK4m5nlkMPdzCyHHO5mZjnkcDczyyGHu5lZDjnczcxyyOFuZpZDDnczsxxqM9wl3SxpjaTnC8oGSJonaXm63qNg3SWSVkh6UdK4SjXczMxaVkrPfQYwvlnZxcD8iBgJzE+3kTQamACMSdtcL6l32VprZmYlaTPcI+J3wBvNio8HZqblmcDnCspnRcTmiHgFWAEcXJ6mmplZqTo65r5XRKwGSNd7pvKhwKqCevWpbDuSJkqqk1S3du3aDjbDzMyKKfcHqipSFsUqRsT0iKiJiJrBgweXuRlmZju3job765KGAKTrNam8Hti7oN4w4LWON8/MzDqio+E+Fzg9LZ8O/KqgfIKkvpJGACOBpzrXRDMza69d2qog6Q6gFhgkqR74PjANmC3pLOCPwEkAEbFU0mxgGbAFmBQRWyvUdjMza0Gb4R4Rp7Sw6sgW6k8FpnamUWZm1jn+hqqZWQ453M3McsjhbmaWQw53M7MccribmeWQw93MLIcc7mZmOeRwNzPLIYe7mVkOOdzNzHLI4W5mlkMOdzOzHGpz4jAzMytiym6d3P4v5WlHC9xzNzPLIYe7mVkOOdzNzHLI4W5mlkMOdzOzHHK4m5nlkMPdzCyHHO5mZjnkcDczyyGHu5lZDjnczcxyyOFuZpZDDnczsxzq1KyQklYCG4CtwJaIqJE0ALgTGA6sBL4YEW92rplmZtYe5ei5Hx4R1RFRk25fDMyPiJHA/HTbzMy6UCWGZY4HZqblmcDnKnAfZmbWis6GewAPSVokaWIq2ysiVgOk6z2LbShpoqQ6SXVr167tZDPMzKxQZ3+J6dMR8ZqkPYF5kv5Q6oYRMR2YDlBTUxOdbIeZmRXoVM89Il5L12uAu4GDgdclDQFI12s620gzM2ufDvfcJX0A6BURG9LyUcAPgLnA6cC0dP2rcjTUulhnfx8SKv4bkWbWss4My+wF3C2pYT+3R8QDkp4GZks6C/gjcFLnm2lmZu3R4XCPiJeBA4qUrweO7EyjzMysc/wNVTOzHHK4m5nlkMPdzCyHHO5mZjnkcDczyyGHu5lZDjnczcxyyOFuZpZDDnczsxxyuJuZ5ZDD3cwshxzuZmY55HA3M8shh7uZWQ453M3McsjhbmaWQw53M7MccribmeWQw93MLIc68wPZZj3e8Ivv7fQ+VvYrQ0PMuph77mZmOZSbnrt7aGat82tk5+Keu5lZDjnczcxyyOFuZpZDDnczsxxyuJuZ5VDFzpaRNB64BugN3BQR0yp1X9aUz4ows4qEu6TewHXAPwD1wNOS5kbEskrcn5lZe+wMHaBKDcscDKyIiJcj4l1gFnB8he7LzMyaUUSUf6fSicD4iPhquv1lYGxEnFNQZyIwMd38CPBi2RvSfoOAdd3diB7Cx2IbH4ttfCy26QnHYp+IGFxsRaXG3FWkrMm7SERMB6ZX6P47RFJdRNR0dzt6Ah+LbXwstvGx2KanH4tKDcvUA3sX3B4GvFah+zIzs2YqFe5PAyMljZD0PmACMLdC92VmZs1UZFgmIrZIOgd4kOxUyJsjYmkl7qvMetQwUTfzsdjGx2IbH4ttevSxqMgHqmZm1r38DVUzsxxyuJuZ5dAOHe6SQtItBbd3kbRW0j3p9nGSLm5h240VatMZkj5YiX03u5+QdFXB7QslTWnnPmolfarg9oz0HYWK6Kpj00Ybhkn6laTlkl6SdI2k90mqlnRMQb0pki7szrZWmqRLJS2VtETSYkljO7CPLn0OVZKkqyWdX3D7QUk3Fdy+StLkduyvybHpajt0uAP/BXxMUv90+x+AVxtWRsTcbpjT5gygKwJsM3CCpEEd2VjSLkAt0JVPvjPommNTlCQBdwFzImIksC9QBUwFqoFjWt663ffVu1z7qgRJhwD/CzgwIvYH/h5Y1YFd1dK1z6FKepz0WCT1IvuS0piC9Z8CHmvH/mrpxmOzo4c7wP3AsWn5FOCOhhWpp3htWh4haaGkpyX9sKDO9ZKOS8t3S7o5LZ8l6fK0fJqkp1Lv5gZJvdNlhqTnJT0n6YLUY6kBbkt1G950KmEL2af1FzRfIWkfSfNTj2y+pL9N5TMk/UjSw8CdwD8BF6S2Hpo2P0zS45JeLuyBSfp2OnZLJP1LQfkcSYtSD3BiKuvuY9OSI4B3IuJnABGxlez4fRW4Ejg5te3kVH+0pAXpWJzbsJNiz4dUvlHSDyQ9CRzSpY+s/YYA6yJiM0BErIuI1yQdKenZ9He7WVJfAEkrGzoSkmrScRlOO55DO4DH2BbGY4DngQ2S9kjHYRSApN+m5/yDkoaksnMlLUuvj1mtHJuuExE77AXYCOwP/ALoBywme7e8J60/A7g2Lc8FvpKWJwEb0/IE4P+k5aeAJ9Lyz4BxZH/QXwN9Uvn1wFeAg4B5BW3ZPV0vAGq66LH/DbAS2A24EJiS1v0aOD0tn0nWUwWYAdwD9E63pwAXFuxzBvBzsjf90WTzAwEcRfZGorTuHuCwtG5Auu5P9mIY2N3HppVjdi5wdZHyZ9O6awvKppD15PqS9eDWA31aej6k5QC+2N2vixKPRVV6vfxHegz/M72GVgH7pjr/DpyfllcCg9JyDbCgPc+hHeWSHuffAl8jC+cfkv1H92lgYXpODE51TyY7zRuyL2n2TcsNz/cmx6arLzt8zz0ilgDDyXrt97VS9dNs69XfUlD+CHCopNHAMuD19G58CNkf8kiysHpa0uJ0+++Al4G/k/T/lE1v/Fa5HlOpIuItshfguc1WHQLcnpZvAT5TsO7nkfVYWzInIt6LbAbPvVLZUenyLPAM8FFgZFp3rqTfA0+QfSt5JD3g2LRANJsGo43yeyNic0SsA9aQHY+Wng8AW4FflrvRlRARG8kex0RgLdl/cl8DXomI/0jVZgKHdWD3xZ5DO4qG3vunyMJ8YcHtV4GPAfPS3/4ysm/fAywh+6/0NLL/qrtdxeZz72JzgX8l67UPbKXedi/giHhV0h7AeOB3wADgi2Q9+w1pnHZmRFzSfFtJB5D17ielbc7s5OPoiH8jC9yftVKn8HH/Vxv721ywrILr/x0RNxRWlFRLNlZ7SES8LWkB0C8i3uwhx6a5pcAXCgsk/Q3Zm1KxN7zCY7GV7PXS4vOBbMintTfOHiW1dQGwQNJzwOmtVN/CtmHctia7LfYc2lE0jLvvR/af6CrgW2QdlP8PDI2IYkNux5K9ER4HfFfSmCJ1utQO33NPbgZ+EBHPtVLnMbIhGIBTm61bCJxPFu6PkA1xPJLWzQdOlLQngKQBaUx7ENArIn4JfBc4MNXfAOzauYdTuoh4A5gNnFVQ/DhNH+ujLWxealsfBM6UVAUgaWg6HrsBb6Zg/yjwybS+RxybIuYD75f0FWj80PMqsqGE1ymtbUWfD5VpbuVI+oikkQVF1WTHYLikD6eyLwO/TcsryXr60PQNsrv/puX2GNkHzW9ExNb0+tqd7L/hO4HByj6MRlIfSWOUffi6d0Q8DHwn1a+im49NLsI9Iuoj4po2qp0HTJL0NFkoFXoE2CUiVpD1ggekMtK/lpcBD0laAswj+zBqKFmPZzFZODT05GYAP+niDw2vIhsXbnAu8I+pvV8me+zF/Br4fFsf+ETEQ2TDPAtTD+8XZE/aB4Bd0v38kGxoBnrWsWkU2UDo54GTJC0nG29+B/hn4GGyD1ALP1Atto+Wng87mipgZsOHgGTj4xcD/wj8PP2d3wN+kur/C3CNpEdo+l9OSc+hHchzZK+lJ5qV/SUi1gAnAlekocjFZL383sCt6Zg9S/a5zp/p5mPj6QfMzHIoFz13MzNryuFuZpZDDnczsxxyuJuZ5ZDD3cwshxzuZmY55HC3HkfS1nRu8POSfi1p91Q+XNKmtK7h0vCFpCpJP1Y2je+zaWKnswu2e75g/59RNvHXH9JlYsG6KZLebviSUiprdXpolTD9sqTfS7qjWdmMdF+7FpRdk/bXMElXw7FouBSdwtqsOYe79USbIqI6Ij4GvEE2hUGDl9K6hsu/p/KbgDeBkRHxcbLpJAY037Gk/0H2hax/ioiPks278zVJxxZUW0f2lfNStTr9sqRRZK+1wyR9oNnqFcDxqV4v4HAKpq1m27FouHT1FNa2g3K4W0+3kOwbry2S9CHgYOCyiHgPICLWRsQVRapPAmZExDOp3jqyr4wX9ohvJpv+d7s3hxa0OP1y8iWyCdweIpt7pNAdZLMLQjY30mP0kImnbMfmcLceK839ciTZxHANPtRsmOJQsrm3f98Q7G0YAyxqVlZH0x9l2EgW8C1N21DMdcCpkppPbQFZeN9JFuSnNFu3nGy+kj3SulnN1vdv9nhbnBrBrFBeZoW0fOmf5qUZThbE8wrWvRQR1YWVlX5speD2pcBJwJ4R0fyXn1qa3rd52f8FFheOpbcmIt6S1DD98qaCtnwCWBsR/ympHrhZ0h4R8WbB5neRTfQ2lmza3UKbmj9es1K45249UUOg7QO8j6Zj7sUsAw5IY9ZExNS0/d8UqbuU7McmCh2U9tEoTfx0O/CNdrT738hm5ywcVz8F+KiklcBLqU1faLbdLLKJ1+aV+N+HWZsc7tZjRcRfyHrCF0rq00q9FWRDK5dr20/e9aP4XOLXAWdIqk71BgJXkP3MXnM/IutJl/QfbvPpl9ObzUnA/hExPCKGk314ekqz7f4IXEr2i0hmZeFwtx4tIp4Ffs+2+embj7k3/ArVV8l+qGWFpEXAb4CLiuxvNXAacKOkP5DNfX9zRPy6SN11wN1kP7VXqsLplw8DXo2IwrNffkc2tXCTaYIj4oaIeKnI/pqPuftsGSuJp/w1M8sh99zNzHLIZ8uYlSCNzc8vsurIiFjf1e0xa4uHZczMcsjDMmZmOeRwNzPLIYe7mVkOOdzNzHLovwFfpoezzNHhDwAAAABJRU5ErkJggg=='.format(encoded_image)) 
            ])
        ], label='Multiple')
    ]),
])   
])   
    
# end of the html for the row

#----------------------------------------
#callback function
@app.callback(Output('line_chart_state', 'figure'),
             Input('dropdown_state', 'value'))
def update_pie_chart(input_year):
    df_filtered_1 = df_agg_1[df_agg_1['STATE_NAME']==input_year]
    fig_line = px.line(df_filtered_1, x = 'DATA_YEAR', y='TOTAL_INDIVIDUAL_VICTIMS')
    fig_line.update_traces(line_color = 'green')
    return fig_line
#------------------------------------------------


#------------------------------------------------
#callback function for Bar 
@app.callback(Output('bar_chart_race', 'figure'),
             Input('dropdown_race', 'value'))
def update_bar_chart(input_race):
    df_filtered_2 = df_agg_2[(df_agg_2['OFFENDER_RACE'] ==input_race)]
    fig2_bar = px.bar(df_filtered_2, x = 'STATE_NAME', y='TOTAL_OFFENDER_COUNT',
                     color_discrete_sequence=px.colors.qualitative.Antique)
    return fig2_bar
#------------------------------------------------

# two input one output
#----------------------------------------
#callback function for two input one output
@app.callback(Output('bar_chart_state', 'figure'),
             [Input('dropdown_state1', 'value'), Input('Victims_slider', 'value')])
def update_bar_chart(victim_state, race_victim):

    df_filtered = df_agg_2.loc[(df_agg_2['STATE_NAME'] ==victim_state)& (df_agg_2['TOTAL_OFFENDER_COUNT']>=(race_victim))]
    fig_bar = px.bar(df_filtered, x ='OFFENDER_RACE', y ='TOTAL_OFFENDER_COUNT', color="OFFENDER_RACE")
    return fig_bar
#----------------------------------------
# callback for one input two output

@app.callback(Output('heatmap_count', 'figure'),
              Output('hist_offender', 'figure'),
              Input('chk_SN', 'value'))
def update_heatmap(options_SN):
    df_filtered3 = df_agg_1.loc[df_agg_1['MULTIPLE_OFFENSE'].isin(options_SN)]
    df_filtered4 = df_agg_2.loc[df_agg_2['MULTIPLE_OFFENSE'].isin(options_SN)]
    fig_bar_s = px.bar(df_filtered3, x = 'STATE_NAME', y = 'TOTAL_INDIVIDUAL_VICTIMS', color="DATA_YEAR")
    #fig_line_s = px.line(df_agg_2, x = 'OFFENDER_ETHNICITY', y = 'TOTAL_OFFENDER_COUNT', color="OFFENDER_RACE")
    
    my_color_scale = [(1, '#629B02'), (0.4, 'rgb(255,165,0)'), (1, '#7a214c')]
    fig_heat = px.density_heatmap(df_filtered4, x = 'OFFENDER_RACE', y = 'TOTAL_OFFENDER_COUNT',
                         color_continuous_scale = my_color_scale)

    return fig_bar_s, fig_heat

# callback for radio buttons
@app.callback(
    Output('state_dpdn', 'options'),
    Input('race_rdb', 'value')
)
def set_state_options(chosen_race_rdb):
    df_filtered5 = df_newexcel[df_newexcel['OFFENDER_RACE']==chosen_race_rdb] # filter on category chosen by radio buttons
    dpdn_state = sorted(df_filtered5['STATE_NAME'].unique()) # create a list of sub-categories corresponding to the
    
    return [{'value':i, 'label':i} for i in dpdn_state]
# populate on initial value for sub-category dropdown box
@app.callback(
    Output('state_dpdn', 'value'),
    Input('state_dpdn', 'options')
)
def set_state_initial_value(available_options):
    return available_options[0]['value']
    
#callback for pie charts

#______________________________________

@app.callback(
    Output('pie-chart', 'figure'),
    [Input('race_rdb', 'value'),
     Input('state_dpdn', 'value')])
def update_pie_chart(sel_race, sel_state):
    
# filter the dataset based on the values chosen by the user

    df_filtered5 = df_newexcel[(df_newexcel['OFFENDER_RACE']==sel_race)& (df_newexcel['STATE_NAME']==sel_state)]

# create the aggregated data
    df_aggregated = df_filtered5.groupby(['DATA_YEAR'], as_index=False).agg({'ADULT_OFFENDER_COUNT':'sum'})

#create the pie chart showing the different segment as slices
    fig_pie = px.pie(df_aggregated, values = 'ADULT_OFFENDER_COUNT', names = 'DATA_YEAR')
    #fig = px.pie(df_newexcel, values='JUVENILE_OFFENDER_COUNT', names='OFFENDER_RACE', title='Juvenile offender per race')
    return fig_pie



app.layout = html.Div([row])
if __name__ == "__main__":
    app.run_server(debug = False)

<ipython-input-1-457372cd4ea4>:10: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Jan/2023 21:46:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:21] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:21] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:21] "GET /_favicon.ico?v=2.6.2 HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:21] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:21] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:21] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:21] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:21] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\preet\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\preet\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\preet\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\preet\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\preet\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\preet\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\preet\anaconda3\lib\site-packages\dash\dash.py", line 1261, in dispatch
    ctx.run(
  Fil

127.0.0.1 - - [04/Jan/2023 21:46:21] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [04/Jan/2023 21:46:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 21:46:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.